<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-1/Insumos/Delta_Normal_Swaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 162 (delta 11), reused 0 (delta 0), pack-reused 138
Receiving objects: 100% (162/162), 1.30 MiB | 1.43 MiB/s, done.
Resolving deltas: 100% (69/69), done.


#Cargar librerias

In [3]:
%%R
library(dplyr)
install.packages("data.table")
require(data.table)
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://cran.rstudio.com/src/contrib/data.table_1.12.6.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/

# Definición de SWAP

Conocido como IRS (interest rate swap), donde por un lado se paga tasa fija y por otro se paga tasa variable en los mismos t\'erminos monetarios, utilizando dos curvas: una para traer a valor presente los flujos y otra para calcular el cup\'on de la tasa variable.
Primero estableceremos la f\'ormula de valoraci\'on de un contrato swap:
$$\textrm{IRS}=\textrm{M}\cdot  (-1)^z\cdot\sum_{i=1}^n{\frac{(\textrm{t}_{c_{p_{i}}}-\textrm{t}_f)\cdot p_{c_i}/360}{\big(1+\textrm{t}_{vp_{p_{i}}}\cdot p_i/360\big)}}$$
	Donde:\ssj
$\textrm{IRS}$: Es el valor del swap de tasa de interes.\
$\textrm{M}$: Es el valor a pagar del flujo.\
$\textrm{z}$: Valor dummy ``0" si paga tasa fija ``1" si paga tasa variable.\
$\textrm{t}_{c_{p_{i}}}$: Tasa cup\'on variable a al plazo $p_i$.\
$\textrm{t}_f$: Tasa fija.\
$p_{c_i}$: Plazo del $i$-\'esimo cup\'on (para este curso $p_{c_i}=p_{c_j}$ para todo $i,j=1,\ldots,n$.)\
$\textrm{t}_{vp_{p_{i}}}$: Tasa valor presente al plazo $p_i$. \
$p_i$: Plazo en dias del $i$-\'esimo cup\'on. \
$n$: N\'umero de cupones a pagar. \


#Declaración de variables

In [0]:
%%R
btasadesc="RiesgosFinancieros/2020-1/Insumos/tasa_TIIE_SW_OP.txt"
btasacupvar="RiesgosFinancieros/2020-1/Insumos/tasa_DIRS_SW_OP.txt"
tasafija=cbind(0.079,0.075) #se establece la tasa fija a pagar para cada swap
plazos=cbind(588,360) #se establece el número de días que vivirá el swap
plazocupon=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos=cbind(-1600,100000) #se establece el número de contratos de cada swap
nominal=cbind(1,1) #se establece el nominal de cada swap
por=cbind(0,0) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable
itpl=1 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada


In [5]:
%%R
plazosmuestra = 377 #@param {type:"integer"}
#@markdown 
plazosmuestra

[1] 377


In [0]:
tasafijaK = 0.08 #@param {type:"slider", min:0.06, max:0.09, step:0.01}
#@markdown 


# Carga de datos

In [0]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc)
n1<-nrow(data1)
m1_orig=ncol(data1)
X1_orig=data.frame(data1[2:n1,2:m1_orig])
nodos1=data.frame(data1[1,2:m1_orig])

data2<-read.table(btasacupvar)
n2<-nrow(data2)
m2_orig=ncol(data2)
X2_orig=data.frame(data2[2:n1,2:m2_orig])
nodos2=data.frame(data2[1,2:m2_orig])


## Transformación de insumos para valoración

In [0]:
%%R      

nodosvp=nodos1
nodostc=nodos2
curvavp=as.matrix(X1_orig/100)
curvatc=X2_orig/100
n1=nrow(curvavp)
n2=nrow(curvatc)
n=max(n1,n2)

m=max(ncol(plazos),1) #número de contratos swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos[j]/plazocupon[j])+1 #número de cupones a pagar
}
VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
porT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos
plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
VTplazosc=matrix(0,1,sum(N)) #vector de todos los plazos cortos de todos los contratos


plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    VTplazosc[,1:sum(N[1:j])]=c(0,VTplazos[,1:(sum(N[1:j])-1)])
    contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
    nominalT[,1:sum(N[1:j])]=seq(nominal[j],nominal[j])
    porT[,1:sum(N[1:j])]=seq(por[j],por[j])
    plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,1:sum(N[1:j])]=seq(tasafija[j],tasafija[j])
  }
  else
  {
    VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    VTplazosc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])					
    nominalT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal[j],nominal[j])
    porT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por[j],por[j])
    plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija[j],tasafija[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos))
Xtc=matrix(0,n,ncol(VTplazos))
Xtcc=matrix(0,n,ncol(VTplazos))
XtfwdT=matrix(0,n,ncol(VTplazos))

#Interpolación

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos)$y}else{talamb(nodosvp,curvavp[i,],VTplazos)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos)$y}else{talamb(nodostc,curvatc[i,],VTplazos)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazosc)$y}else{talamb(nodostc,curvatc[i,],VTplazosc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos/360)/(1+Xtcc[i,]*VTplazosc/360)-1)*360/plazocuponT 
  for (j in (1:ncol(VTplazos)))
  {
    if (VTplazos[j]<= plazocuponT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

Primero estableceremos la f\'ormula de valoraci\'on de un contrato swap:
$$\textrm{IRS}=\textrm{M}\cdot  (-1)^z\cdot\sum_{i=1}^n{\frac{(\textrm{t}_{c_{p_{i}}}-\textrm{t}_f)\cdot p_{c_i}/360}{\big(1+\textrm{t}_{vp_{p_{i}}}\cdot p_i/360\big)}}$$
	Donde:\ssj
$\textrm{IRS}$: Es el valor del swap de tasa de interes.\
$\textrm{M}$: Es el valor a pagar del flujo.\
$\textrm{z}$: Valor dummy ``0" si paga tasa fija ``1" si paga tasa variable.\
$\textrm{t}_{c_{p_{i}}}$: Tasa cup\'on variable a al plazo $p_i$.\
$\textrm{t}_f$: Tasa fija.\
$p_{c_i}$: Plazo del $i$-\'esimo cup\'on (para este curso $p_{c_i}=p_{c_j}$ para todo $i,j=1,\ldots,n$.)\
$\textrm{t}_{vp_{p_{i}}}$: Tasa valor presente al plazo $p_i$. \
$p_i$: Plazo en dias del $i$-\'esimo cup\'on. \
$n$: N\'umero de cupones a pagar. \

#Función de valoración

In [9]:
%%R                

swap=function(porT, contratosT, nominalT, XtfwdT, tasafijaT, plazocuponT, VTplazos, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratosT*(XtfwdT-tasafijaT)*(plazocuponT/360)))/(1+Xvp*VTplazos/360))*nominalT*(-1)^porT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0=swap(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT, plazocuponT, VTplazos, Xvp[1,], N)
V0

         [,1]     [,2]
[1,] 6.143429 442.5273


# Cálculo de $\Delta X$ y Var-Cov

Delta Normal

In [10]:
%%R     
DeltaXvp=Xvp[1:(n-1),]/Xvp[2:n,]-1
DeltaXtfwdt=XtfwdT[1:(n-1),]/XtfwdT[2:n,]-1

DeltaX=cbind(DeltaXtfwdt,DeltaXvp)

VarCovX=cov(DeltaX)*((n-1)/(n-2))  #Matriz de Varianza-Covarianza
ncol(VarCovX)

[1] 70


# Cálculo de derivadas

In [11]:
%%R
#derivada del swap con respecto aL tiempo
dswap_dtt=function(porT, contratosT, nominalT, XtfwdT, tasafijaT, plazocuponT, VTplazos, Xvp)
{
-1*(((contratosT*(XtfwdT-tasafijaT)*(plazocuponT*Xvp/360^2)))/(1+Xvp*VTplazos/360)^(2))*nominalT*(-1)^porT
}

#derivada del swap con respecto a la tasa cupón variable
dswap_dtc=function(porT, contratosT, nominalT, XtfwdT, tasafijaT, plazocuponT, VTplazos, Xvp)
{
  (((contratosT*plazocuponT)/360)/(1+Xvp*VTplazos/360))*nominalT*(-1)^porT
}

#derivada del swap con respecto al valor presente
dswap_dvp=function(porT, contratosT, nominalT, XtfwdT, tasafijaT, plazocuponT, VTplazos, Xvp)
{
-1*(((contratosT*(XtfwdT-tasafijaT)*(plazocuponT*VTplazos/360^2)))/(1+Xvp*VTplazos/360)^(2))*nominalT*(-1)^porT
}

diagv=function(x)			#función para diagonalizar un vector
{
n01=nrow(as.matrix(x))
m01=ncol(as.matrix(x))
dimmax=max(n01,m01)
		res=matrix(0,dimmax,dimmax)
		for (i in 1:dimmax)
		{
		res[i,i]=x[i]
		}
	res
}

diagv(c(1,2,3))

     [,1] [,2] [,3]
[1,]    1    0    0
[2,]    0    2    0
[3,]    0    0    3


# Medidas de riesgo

## Totales

In [1]:
%%R
#Evalución de las derivada en el punto x0

g=cbind(dswap_dtc(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,]),dswap_dvp(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,]))

#Multiplicación de la derivada por el punto x0
gt=c( XtfwdT[1,],Xvp[1,])*g
#print(g)
#print(gt)
ncol(gt)
VaRTotal=sqrt(gt%*%VarCovX%*%t(gt))*qnorm(.02)#+dswap_dtt(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
VaRTotal



UsageError: Cell magic `%%R` not found.


##Por contrato

In [12]:
%%R

#Valor de las derivadas evaluadas en el punto x0.
alpha = .98
VaRcont=matrix(0,1,ncol(N))
CVaRcont=matrix(0,1,ncol(N))

V0f_dtt=dswap_dtt(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
V0f_dtc=dswap_dtc(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
V0f_dvp=dswap_dvp(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
for (j in (1:ncol(N)))
{
  if(j==1)
  {
    mucont=sum(V0f_dtt[1:sum(N[1:j])])
    gcont=as.matrix(c(V0f_dtc[1:sum(N[1:j])],V0f_dvp[1:sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
    #gamma_cont=gamma[1:(2*sum(N[1:j])),1:(2*sum(N[1:j]))]
    #tgamma= c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])%*%t(c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])]))*gamma_cont #gamma tilde
  }
  else
  {
    mucont=sum(V0f_dtt[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont=as.matrix(c(V0f_dtc[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    #gamma_cont=gamma[(2*sum(N[1:j-1])+1):(2*sum(N[1:j])),(2*sum(N[1:j-1])+1):(2*sum(N[1:j]))]
    #tgamma= c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])%*%t(c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]))*gamma_cont #gamma tilde
  }
  
  
  #tgammavar=(tgamma%*%VarCovXcont)
  #tgammavar2=(tgamma%*%VarCovXcont)%*%(tgamma%*%VarCovXcont)
  #VaRContDGM[j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*qnorm(alpha) #VaR Normal
  #CVaRContDGM[j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(qnorm(alpha))/(1-alpha)
  ###VaR Cornish Fisher
  #tgammavar3=tgammavar2%*%(tgamma%*%VarCovXcont)
  #asimPmargc=sum(diag(tgammavar3))+3*gtcont%*%VarCovXcont%*%tgamma%*%VarCovXcont%*%t(gtcont) #gamma tercer momento
  #asimPmargc=asimPmargc/(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))^3  #coef de asimetría
  #VaRmargcstd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc  #VaR estandarizado
  #VaRContCF[j]=(sum(diag(tgammavar))/2+mucont)+(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*VaRmargcstd #VaR Normal
  #CVaRContCF[j]=(sum(diag(tgammavar))/2+sum(mucont))-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(VaRmargcstd)/(pnorm(VaRmargcstd))
  
  #VaR Delta Normal
  VaRcont[j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*qnorm(alpha)
  CVaRcont[j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*dnorm(qnorm(alpha))/(1-alpha)
}

VaRcont

          [,1]      [,2]
[1,] -2.285795 -81.16561


##Por contrato-factor de riesgo subyacente

In [13]:
%%R
#Para obtener el VaR por factor de riesgo
#creación de vector de pesos por factor de riesgo.
WFR=rbind(c( XtfwdT[1,],Xvp[1,]*0),c( XtfwdT[1,]*0,Xvp[1,]))
VaRFr=matrix(0,1,2)
CVaRFr=matrix(0,1,2)

for (j in (1:2))
{
   # gFr=as.matrix(V0f_dtc)
    #gtFr=as.matrix(gFr*XtfwdT[1,])
    #VarCovXFr=cov(cbind(XtfwdT[,]))*(n/(n-1))
    
    g=cbind(dswap_dtc(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,]),dswap_dvp(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,]))
    
    #Multiplicación de la derivada por el punto x0
    gt=WFR[j,]*g
    VaRFr[j]=-sqrt(gt%*%VarCovX%*%t(gt))*qnorm(alpha)
    CVaRFr[j]=-sqrt(gt%*%VarCovX%*%t(gt))*dnorm(qnorm(alpha))/(1-alpha)
}

print(VaRFr)
print(CVaRFr)


          [,1]       [,2]
[1,] -79.44325 -0.1468384
          [,1]       [,2]
[1,] -93.64568 -0.1730894


## Medida de riesgo a nivel contrato-factor de riesgo.

In [14]:
%%R

#Valor de las derivadas evaluadas en el punto x0.
alpha = .98
VaRcontfr=matrix(0,2,ncol(N))
CVaRcontfr=matrix(0,2,ncol(N))
WFRcont <- rbind(c(1,0), c(0,1))

V0f_dtt=dswap_dtt(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
V0f_dtc=dswap_dtc(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
V0f_dvp=dswap_dvp(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT,  plazocuponT, VTplazos, Xvp[1,])
for (j in (1:ncol(N)))
{
    for(k in (1:2)){
  if(j==1)
  {
    mucont=sum(V0f_dtt[1:sum(N[1:j])])
    gcont=as.matrix(c(V0f_dtc[1:sum(N[1:j])],V0f_dvp[1:sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,1:sum(N[1:j])]*WFRcont[k,1],Xvp[1,1:sum(N[1:j])]*WFRcont[k,2])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
    #gamma_cont=gamma[1:(2*sum(N[1:j])),1:(2*sum(N[1:j]))]
    #tgamma= c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])%*%t(c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])]))*gamma_cont #gamma tilde
  }
  else
  {
    mucont=sum(V0f_dtt[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont=as.matrix(c(V0f_dtc[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,1],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,2])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    #gamma_cont=gamma[(2*sum(N[1:j-1])+1):(2*sum(N[1:j])),(2*sum(N[1:j-1])+1):(2*sum(N[1:j]))]
    #tgamma= c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])%*%t(c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]))*gamma_cont #gamma tilde
  }
  
  
  #tgammavar=(tgamma%*%VarCovXcont)
  #tgammavar2=(tgamma%*%VarCovXcont)%*%(tgamma%*%VarCovXcont)
  #VaRContDGM[j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*qnorm(alpha) #VaR Normal
  #CVaRContDGM[j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(qnorm(alpha))/(1-alpha)
  ###VaR Cornish Fisher
  #tgammavar3=tgammavar2%*%(tgamma%*%VarCovXcont)
  #asimPmargc=sum(diag(tgammavar3))+3*gtcont%*%VarCovXcont%*%tgamma%*%VarCovXcont%*%t(gtcont) #gamma tercer momento
  #asimPmargc=asimPmargc/(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))^3  #coef de asimetría
  #VaRmargcstd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc  #VaR estandarizado
  #VaRContCF[j]=(sum(diag(tgammavar))/2+mucont)+(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*VaRmargcstd #VaR Normal
  #CVaRContCF[j]=(sum(diag(tgammavar))/2+sum(mucont))-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(VaRmargcstd)/(pnorm(VaRmargcstd))
  
  #VaR Delta Normal
  VaRcontfr[k,j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*qnorm(alpha)
  CVaRcontfr[k,j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*dnorm(qnorm(alpha))/(1-alpha)
}
}

VaRcont

          [,1]      [,2]
[1,] -2.285795 -81.16561


In [15]:
%%R
VaRcontfr

            [,1]        [,2]
[1,] -2.28597304 -81.1651479
[2,] -0.01214076  -0.2356319
